In [1]:
import torch

In [2]:
a = torch.rand(3,32,8)
b = torch.rand(6,32,8)
c = torch.rand(3,32,8)
print(torch.cat([a,b],dim=0).shape)
print(torch.stack([a,c],dim=0).shape)

torch.Size([9, 32, 8])
torch.Size([2, 3, 32, 8])


In [3]:
a = torch.rand(5,32,8)
b = torch.split(a,2,0)
print(b[0].shape)
print(b[1].shape)
print(b[2].shape)
print(len(b))
print(type(b))

c = torch.chunk(a,2,0)
print(c[0].shape)
print(c[1].shape)
print(len(c))
print(type(c))

torch.Size([2, 32, 8])
torch.Size([2, 32, 8])
torch.Size([1, 32, 8])
3
<class 'tuple'>
torch.Size([3, 32, 8])
torch.Size([2, 32, 8])
2
<class 'tuple'>


In [4]:
a = torch.tensor(3.1415926)
print(a.floor())
print(a.ceil())
print(a.round())
print(a.trunc())
print(a.frac())


tensor(3.)
tensor(4.)
tensor(3.)
tensor(3.)
tensor(0.1416)


In [5]:
a = torch.tensor([1.,2.,3.,4.,5.,6.,7.])
print(a)

print(a.mean())
print(a.max())
print(a.min())
print(a.sum())
print(a.prod())

print(a.argmax())
print(a.argmin())





tensor([1., 2., 3., 4., 5., 6., 7.])
tensor(4.)
tensor(7.)
tensor(1.)
tensor(28.)
tensor(5040.)
tensor(6)
tensor(0)


In [6]:
a = torch.ones(3,3)
b = torch.eye(3,3)
print(a)
print(b)

print(torch.eq(a,b))
print(torch.equal(a,b))


tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])
tensor([[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]])
tensor([[ True, False, False],
        [False,  True, False],
        [False, False,  True]])
False


In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim

e:\Anaconda\envs\Torch\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'

In [3]:
class Net(nn.Module):
    #定义网络层
    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        self.fc1 = nn.Linear(784,100)
        self.fc2 = nn.Linear(100,5)
        self.fc3 = nn.Linear(5,10)
    #hook
    #网络框架
    def forward(self,x):
        x = torch.flatten(x,start_dim =1)
        x = torch.relu(self.fc1(x))
        x=  torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

max_epochs = 5
batch_size = 16

#data
transform = transforms.Compose([transforms.ToTensor()])
#数据集下载
trainset = torchvision.datasets.MNIST(root='./data',train=True,download=True,transform=transform)
train_loader = torch.utils.data.DataLoader(trainset,batch_size=batch_size,shuffle =True)
testset = torchvision.datasets.MNIST(root='./data',train=True,download=False,transform=transform)
test_loader = torch.utils.data.DataLoader(testset,batch_size=batch_size,shuffle =True)

#net init
net = Net()
net.to(device)

#nn.MSE
loss = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(),lr = 0.001)

def train():
    acc_num = 0
    for epoch in range(max_epochs):
        for i ,(data,label) in enumerate(train_loader):
            data = data.to(device)
            label = label.to(device)
            optimizer.zero_grad()
            output = net(data)
            Loss = loss(output,label)
            Loss.backward()
            optimizer.step()

            pred_class = torch.max(output,dim=1)[1]
            acc_num += torch.eq(pred_class,label.to(device)).sum().item()
            train_acc = acc_num/len(trainset) 
        
        net.eval()
        acc_num =0.0
        best_acc = 0
        with torch.no_grad():
            for val_data in test_loader:
                val_image,val_label = val_data
                output = net(val_image.to(device))
                predict_y = torch.max(output,dim=1)[1]
                acc_num += torch.eq(predict_y,val_label.to(device)).sum().item()
            val_acc = acc_num/len(testset) 
            print(train_acc,val_acc)
            if val_acc > best_acc:
                torch.save(net.state_dict(),'./minst.pth')
                best_acc = val_acc
            acc_num = 0
            train_acc = 0
            test_acc = 0
    print("done")


In [6]:
train()

0.8641666666666666 0.8672333333333333
0.8704 0.8733666666666666
0.8760666666666667 0.88025
0.8814333333333333 0.88505
0.8867166666666667 0.8891
done
